# Assignment 3 - Transfer Learning
 
**Authors:**

1.   Liav Bachar 205888472
2.   Naor Kolet 205533060


# 0. Imports

In [32]:
import pandas as pd
import numpy as np



# TensorFlow
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import VGG16, ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Scikit-learn
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, accuracy_score
# from sklearn.model_selection import StratifiedKFold
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import log_loss
 
# Plots
# import seaborn as sns
import matplotlib.pyplot as plt

# Misc.
from scipy.io import loadmat
import os
import random
import joblib
import cv2
from glob import glob
from tqdm import tqdm_notebook as tqdm

%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Download Data

In [3]:
# Another dataset? https://www.tensorflow.org/tutorials/load_data/images
if not os.path.exists(r'./datasets/'):
    !mkdir ./datasets
    !wget 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz' -P './datasets/'
    !tar -xf ./datasets/102flowers.tgz -C ./datasets/
    
#     !wget 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102segmentations.tgz' -P './datasets/'
    
    !wget 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat' -P './datasets/'
    
labels = loadmat('./datasets/imagelabels.mat')['labels'].reshape(-1)

## Load Images

In [6]:
images_path = glob('./datasets/jpg/*')

In [14]:
def split_train_val_test(split_seed):
    # train 50% validation 25% test 25%
    train_paths, val_tst_paths, train_labels, val_tst_labels = train_test_split(images_path, labels, train_size=0.5, random_state=split_seed, shuffle=True, stratify=labels)
    val_paths, tst_paths, val_labels, tst_labels = train_test_split(val_tst_paths, val_tst_labels, train_size=0.5, random_state=split_seed, shuffle=True, stratify=val_tst_labels)
    
    def convert2df(paths, labels): return pd.DataFrame({'filename': paths, 'class':labels}).astype(str)
    train_df = convert2df(train_paths, train_labels)
    val_df = convert2df(val_paths, val_labels)
    test_df = convert2df(tst_paths, tst_labels)
    
    return train_df, val_df, test_df

In [15]:
train_df, val_df, test_df = split_train_val_test(split_seed=SEED)

## Image Data Generator

In [70]:
def datagen_flow(df, batch_size, resize_shape, subset):
    classes = list(np.unique(labels).astype(str))
    
    if subset == 'trainy':
        data_gen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            rescale=1./255)
    else:
        data_gen = ImageDataGenerator(rescale=1./255)
        
    return data_gen.flow_from_dataframe(df, batch_size=batch_size, target_size=resize_shape, seed=SEED, validate_filenames=False, classes=classes)

In [58]:
classes = list(np.unique(labels).astype(str))
data_loader_train = datagen_flow(train_df, 32, (224,224), 'train')
data_loader_val = datagen_flow(val_df, 32, (224,224), 'val')
data_loader_test = datagen_flow(test_df, 32, (224,224), 'test')

Found 4094 non-validated image filenames belonging to 102 classes.
Found 2047 non-validated image filenames belonging to 102 classes.
Found 2048 non-validated image filenames belonging to 102 classes.


In [65]:
a, b = data_loader_train.next()
a.shape, b.shape

((32, 224, 224, 3), (32, 102))

# Pre-trained models

In [24]:
def get_feature_extractor(base_model, width=224, height=224, channel=3):
    model = base_model(input_shape=(width, height, channel), include_top=False)
    
    for layer in model.layers:
        layer.trainable = False
    
    return Model(model.input, model.output, name=base_model.__name__)


In [149]:
vgg_fe = get_feature_extractor(VGG16)
resent_fe = get_feature_extractor(ResNet50V2)

# Building the base model

## Model Architecture

<!-- Here we build our model architecture. It contains 4 main parts.

1. **Embedding Layer** - Here the words of each song will be converted to their embeddings representations. We made this layer trainable, let the model learn the embeddings for the missing words.
2. **Concatenate Layer** - Here we attach for each word entry the related melody record
3. **LSTM Layers** - Here we created two layers of lstm, the first one learn features across the windows and outputs all the hidden states, the next one gets those hidden states and output a single feature that was learnt from them.
4. **Softmax Layer** - Give each word in our dictionary the probabilty it will be the next word after the window.

Note: We are using Dropout layers in order to reduce the overfitting -->

In [41]:
def init_model(image_shape, output_shape, fe_type='vgg'):
    inp = Input(shape=image_shape, name='image')
    
    if fe_type == 'vgg':
        X = get_feature_extractor(VGG16)(inp)
    elif fe_type == 'resnet':
        X = get_feature_extractor(ResNet50V2)(inp)
    else:
        X = inp

    X = Conv2D(256, 3, activation='relu', padding='same')(X)
    X = Conv2D(128, 3, activation='relu', padding='same')(X)
    X = Flatten()(X)
    X = Dense(4096, activation="relu")(X)
    X = Dense(2048, activation="relu")(X)
    X = Dense(1024, activation="relu")(X)
    out = Dense(output_shape, activation="softmax", name = 'out')(X)

    model = Model(inp, out)

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    
    return model

In [43]:
image_shape = (224, 224, 3)
output_shape = len(classes)
model = init_model(image_shape, output_shape, fe_type='resnet')
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
ResNet50V2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 256)         4718848   
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 128)         295040    
_________________________________________________________________
flatten_9 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 4096)              25694208  
_________________________________________________________________
dense_22 (Dense)             (None, 2048)              8390

## Training Framework

In [46]:
def get_callbacks(model_name):
    acc = 'val_loss'
    acc_mode = 'min'
    
    checkpoint = ModelCheckpoint(
                              fr'./models/{model_name}.h5', 
                              monitor=acc, 
#                               verbose=1, 
                              save_best_only=True, 
                              mode=acc_mode)
    earlystop = EarlyStopping(monitor=acc, mode=acc_mode, verbose=1, patience=6)
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 5,
                            factor = 0.5, min_lr = 1e-6, verbose = 1)

    return [reduceLR, earlystop] #checkpoint, 

In [73]:
def train_model(model_gen, fe_type, train_df, val_df, use_saved=False, params_dict=None):
    os.makedirs('./models', exist_ok=True)
    params = ''
    if params_dict is not None:
        params = '_'.join(f'{key}_{val}' for key,val in params_dict.items())
    model_name = f'{fe_type}' + f'_{params}'
        
    if use_saved:
        history = joblib.load(fr'./models/{model_name}_history.sav')
    else:
        callbacks = get_callbacks(model_name)
        
        train_flow = datagen_flow(train_df, params_dict['batch_size'], image_shape[:-1], 'train')
        val_flow = datagen_flow(val_df, params_dict['batch_size'], image_shape[:-1], 'val')
        
        model = model_gen(image_shape, output_shape, fe_type)
        history = model.fit(
                            x=train_flow,
                            y=None,
                            batch_size=params_dict['batch_size'],
                            epochs=params_dict['epochs'],
                            validation_data=val_flow,
                            callbacks=callbacks,
                            steps_per_epoch = params_dict['steps'],
                            validation_steps = params_dict['steps'],
                            )
        
        history = history.history
        joblib.dump(history, fr'./models/{model_name}_history.sav')
    
#     model = load_model(fr'./models/{model_name}.h5')
    
    return model, history

In [68]:
def visualize_perf(history):
    ncols = 1
    fig, ax = plt.subplots(ncols=ncols, figsize=(5*ncols,5))
    fig.suptitle(f'Model performance over epochs')
    
    for k in ['val_loss', 'loss']:
        ax.plot(history[k])
        
    ax.legend(['val_loss', 'train_loss'])
    ax.margins(0.01)
    ax.set_title('binary crossentropy')
    
    plt.show()

## Model Learning

In [ ]:
params_dict = {
    'epochs': 20,
    'batch_size': 32,
    'steps': 256
}

vgg_model, vgg_history = train_model(init_model, 'resnet', train_df, val_df, params_dict=params_dict)

Found 4094 non-validated image filenames belonging to 102 classes.
Found 2047 non-validated image filenames belonging to 102 classes.
Epoch 1/20
